In [ ]:
%load_ext snakeviz
%load_ext autoreload
%autoreload 2
from scipy.io import wavfile
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa
import librosa.display
from scipy import sparse
from letitbee import *
from particle import *
from probutils import *
import time

# Let It Bee

In [ ]:
win = 2048
hop = 1024
p = 10

y1, sr = librosa.load("aha.mp3")
y2, sr = librosa.load("theoshift.mp3")
#y1, sr = librosa.load("Beatles_LetItBe.mp3")
#y2, sr = librosa.load("Bees_Buzzing.mp3")

W1 = get_windowed(y1, hop, win)
V = np.abs(np.fft.fft(W1, axis=0)[0:win//2+1, :])
WSound = get_windowed(y2, hop, win)
W = np.abs(np.fft.fft(WSound, axis=0)[0:win//2+1, :])

In [ ]:
c = 3 # Diagonal
r = 3 # Repeated activation
tic = time.time()
HGT = get_musaic_activations(V, W, L=50, r=r, p=p, c=c, verbose=False)
print("Elapsed time ground truth: {:.3f}".format(time.time()-tic))
WH = W.dot(HGT)
gt_fit = np.sum(V*np.log(V/WH) - V + WH)
y = do_windowed_sum(WSound, HGT, win, hop)
ipd.Audio(y, rate=sr)

In [ ]:
pd = 0.99
sigma = 0.1
L = 10
P = 2000
gamma = 0
neff_thresh = 0.1*P
tic = time.time()
tic = time.time()
H, wsmax, neff = get_particle_musaic_activations(V, W, p, pd, sigma, L, P, gamma=gamma, c=c, neff_thresh=neff_thresh, use_gpu=True)
print("Elapsed time particle filter: {:.3f}".format(time.time()-tic))

a1 = get_activations_diff(HGT, p)
a2 = get_activations_diff(H, p)
r1 = get_repeated_activation_itervals(HGT, p)
r2 = get_repeated_activation_itervals(H, p)

WH = W.dot(H)
fit = np.sum(V*np.log(V/WH) - V + WH)
y = do_windowed_sum(WSound, H, win, hop)
ipd.Audio(y, rate=sr)

In [ ]:
y = y/np.max(y)
prefix = "p{}_pd{}_sigma{}_L{}_c{}_P{}_neff{}".format(p, pd, sigma, L, c, P, neff_thresh)
wavfile.write("{}.wav".format(prefix), sr, y)

plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.plot(a1)
plt.plot(a2)
plt.legend(["Ground Truth: Mean {:.3f}".format(np.mean(a1)),
            "Particle Filter: Mean {:.3f}".format(np.mean(a2))])
plt.title("Activation Changes over Time")
plt.xlabel("Timestep")
plt.subplot(232)
plt.hist(a1, bins=np.arange(p+2), alpha=0.5)
plt.hist(a2, bins=np.arange(p+2), alpha=0.5)
plt.title("Activation Changes Histogram")
plt.xlabel("Number of Activations Changed")
plt.ylabel("Counts")
plt.legend(["Ground Truth", "Particle Filter"])
plt.subplot(233)
plt.hist(r1, bins=np.arange(c*20), alpha=0.5)
plt.hist(r2, bins=np.arange(c*20), alpha=0.5)
plt.title("Repeated Activations Histogram, c={}".format(c))
plt.xlabel("Repeated Activation Distance")
plt.ylabel("Counts")
plt.legend(["Ground Truth", "Particle Filter"])
plt.subplot(234)
plt.plot(wsmax)
plt.title("Max Probability, Overall Fit: {:.3f}\nGround Truth Fit: {:.3f}".format(fit, gt_fit))
plt.xlabel("Timestep")
plt.subplot(235)
plt.plot(neff)
plt.xlabel("Timestep")
plt.title("Number of Effective Particles")
plt.subplot(236)
diagsgt = get_diag_lengths(HGT, p)
diags = get_diag_lengths(H, p)
plt.hist(diagsgt, bins=np.arange(30), alpha=0.5)
plt.hist(diags, bins=np.arange(30), alpha=0.5)
plt.legend(["Ground Truth Mean: {:.3f} (c={})".format(np.mean(diagsgt), c),
            "Particle Filter Mean: {:.3f} ($p_d$={})".format(np.mean(diags), pd)])
plt.xlabel("Diagonal Length")
plt.ylabel("Counts")
plt.title("Diagonal Lengths")
plt.tight_layout()
plt.savefig("{}.svg".format(prefix), bbox_inches='tight')


